In [1]:
import pandas as pd
import numpy as np
from itertools import combinations


class DatasetManipulator:
    def __init__(self, excel_path):
        self.excel_path = excel_path
        self.data = pd.read_excel(self.excel_path)
        self.data = self.data[['ExhibitionID', 'ConstituentID']]

        # Step 1: Create pairs of artists for each exhibition
        self.pairs = (
            self.data.groupby('ExhibitionID')['ConstituentID']
            .apply(lambda x: pd.DataFrame(combinations(x, 2), columns=['artist_a', 'artist_b'])
            .assign(weighed_sum=1/len(x))
                   )
            .reset_index(drop=True)
            .groupby(['artist_a', 'artist_b']).weighed_sum.sum()
            .size() 
            .reset_index(name='weight')
        )
                 
    def write_dot_files(self):
        result = (
            self.pairs.groupby(['artist_a', 'artist_b'])
            .size() 
            .reset_index(name='weight')
            .assign({'weight_partial': self.pairs
                     .groupby(['artist_a', 'artist_b'])}).weighted_sum.sum()
            .sum()
        )
#        result.merge(self.pairs)
        
        filters = [
         (result[result['weight'] > 0], "0"),
         (result[result['weight'] > 1], "1"),
         (result[result['weight'] > 5], "5"),
         (result[result['weight'] < 250], "250"),
         (result[(result['weight'] < 250) & (result['weight'] > 1)], "1_250"),
         (result[(result['weight'] < 250) & (result['weight'] > 5)], "5_250")
         ]
        
        for filter_tuple in filters:
            output_dot_file = f'./dot_files/artist_pairs_with_weights{filter_tuple[1]}.dot'
            output_exp_dot_file = f'./dot_files/artist_pairs_with_weights_exp_{filter_tuple[1]}.dot'
            output_partial_weights_dot_file = f'./dot_files/artist_pairs_with_weights_partial_weights_{filter_tuple[1]}.dot'

            with open(output_dot_file, 'w') as f:
                with open(output_exp_dot_file, 'w') as f_exp:
                    with open(output_partial_weights_dot_file, 'w') as f_p:
                        for _, row in filter_tuple[0].iterrows():
                            f.write(f'{row["artist_a"]} {row["artist_b"]} {row["weight"]}\n')
                            f_exp.write(f'{row["artist_a"]} {row["artist_b"]} {np.exp(row["weight"])}\n')
                        for _, row in filter[0][0].iterrows():
                            f_p.write(f'{row["artist_a"].values[0]} {row["artist_b"].values[0]} {row["weight"]["sum"]}\n')
        

dm = DatasetManipulator("./final_dataset.xlsx")
dm.write_dot_files()



/var/folders/0c/kb824dp55mv5r_ksvh54vyqr0000gn/T/ipykernel_31150/2895574676.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  .apply(lambda x: pd.DataFrame(combinations(x, 2), columns=['artist_a', 'artist_b'])


TypeError: 'int' object is not callable

In [3]:
import pandas as pd
import numpy as np
from itertools import combinations

excel_path = "./final_dataset.xlsx"
data = pd.read_excel(excel_path)
data = data[['ExhibitionID', 'ConstituentID']]

# Step 1: Create pairs of artists for each exhibition
pairs = (
    data.groupby(['ExhibitionID'])['ConstituentID']
    .apply(lambda x: pd.DataFrame(combinations(x, 2), columns=['artist_a', 'artist_b'])
    .assign(weighed_sum=1/len(x)))
    .reset_index(drop=True)
)
pairs['weight'] = 1

/var/folders/0c/kb824dp55mv5r_ksvh54vyqr0000gn/T/ipykernel_31150/727822203.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  .apply(lambda x: pd.DataFrame(combinations(x, 2), columns=['artist_a', 'artist_b'])


In [5]:
pairs.weight = 1

In [13]:
final = pairs.groupby(['artist_a', 'artist_b']).agg({'weight': 'sum', 'weighed_sum': 'sum'}).reset_index()

In [15]:
final

,artist_a,artist_b,weight,weighed_sum
0,1.0,11.0,3,0.029703
1,1.0,262.0,1,0.009901
2,1.0,406.0,2,0.026850
3,1.0,412.0,3,0.044118
4,1.0,452.0,1,0.009901
...,...,...,...,...
84650,66686.0,3400.0,1,0.076923
84651,66686.0,3567.0,1,0.076923
84652,66686.0,4945.0,1,0.076923
84653,66730.0,2927.0,1,0.250000


In [20]:
pairs[pairs['artist_a'] == 1.0]

,artist_a,artist_b,weighed_sum,weight
73650,1.0,412.0,0.014706,1
73651,1.0,412.0,0.014706,1
73652,1.0,412.0,0.014706,1
73653,1.0,682.0,0.014706,1
73654,1.0,710.0,0.014706,1
...,...,...,...,...
512577,1.0,5480.0,0.016949,1
512578,1.0,5804.0,0.016949,1
512579,1.0,5821.0,0.016949,1
512580,1.0,6318.0,0.016949,1


In [21]:
final

,artist_a,artist_b,weight,weighed_sum
0,1.0,11.0,3,0.029703
1,1.0,262.0,1,0.009901
2,1.0,406.0,2,0.026850
3,1.0,412.0,3,0.044118
4,1.0,452.0,1,0.009901
...,...,...,...,...
84650,66686.0,3400.0,1,0.076923
84651,66686.0,3567.0,1,0.076923
84652,66686.0,4945.0,1,0.076923
84653,66730.0,2927.0,1,0.250000
